In [1]:
import pandas as pd
import numpy as np
import yaml
from pathlib import Path
from collections import defaultdict
from data_processor import SampleGenerator
from GIMF import GIMFDeepRecommender
import logging

In [2]:
import autoreload
%load_ext autoreload
%autoreload 2

In [3]:
logging.basicConfig()
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

In [4]:
def load_data(data_dir, MIN_RATINGS=4):
    ml_dir = data_dir
    df = pd.read_csv(ml_dir, sep='::', header=None, names=[
        'uid', 'mid', 'rating', 'timestamp'],  engine='python')

    # first let us filter out the users with less than MIN_RATINGS interations
    logger.info(
        "Filtering out users with less than {} ratings".format(MIN_RATINGS))
    grouped = df.groupby('uid')
    df = grouped.filter(lambda x: len(x) >= MIN_RATINGS)
    # print(df.head())

    # no let us factoriyze (re-index users)
    logger.info("Mapping original user and item IDs to new sequential IDs")
    df['userId'] = pd.factorize(df['uid'])[0]
    df['itemId'] = pd.factorize(df['mid'])[0]

    logger.info('Range of userId is [{}, {}]'.format(
        df.userId.min(), df.userId.max()))
    logger.info('Range of itemId is [{}, {}]'.format(
        df.userId.min(), df.itemId.max()))

    num_users = len(df['userId'].unique())
    num_items = len(df['itemId'].unique())
    logger.info("num_users is {}, num_items is {}".format(
        num_users, num_items))

    return df, num_users, num_items

In [5]:
config_dir = Path("./config/")
config_location = config_dir / "config.yml"
c_list = defaultdict()
with open(config_location) as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    c_list = yaml.load(file, Loader=yaml.FullLoader)

logger.info("config {} is loaded".format(c_list))

gmf_config = c_list['gmf']

ml1m_dir = '/data/ml-1m/ratings.dat'
data, num_users, num_items = load_data(ml1m_dir)

# prepare data for training
sample_generator = SampleGenerator(ratings=data)
evaluation_data = sample_generator.evaluation_data
print(len(evaluation_data))
print(len(evaluation_data[0]))
print(type(evaluation_data))

INFO:__main__:config {'gmf': {'alias': 'gmf_imf', 'num_epoch': 200, 'batch_size': 1024, 'optimizer': 'adam', 'adam_lr': '1e-3', 'latent_dim': 8, 'num_negatives': 4, 'l2_regularization': 0.1, 'use_cuda': True, 'model_dir': 'checkpoints/{}_Epoch{}_HR{:.4f}_NDCG{:.4f}.model'}} is loaded
INFO:__main__:Filtering out users with less than 4 ratings
INFO:__main__:Mapping original user and item IDs to new sequential IDs
INFO:__main__:Range of userId is [0, 6039]
INFO:__main__:Range of itemId is [0, 3705]
INFO:__main__:num_users is 6040, num_items is 3706


4
6040
<class 'list'>


In [24]:
test_ratings_df, positive_interactions_users, test_items, negative_interactions_users, negative_items =  sample_generator.evaluation_data

In [25]:
test_ratings_df.head()

,userId,itemId,rating,negative_samples
0,0,25,1,"[2717, 2515, 1722, 1981, 2504, 734, 196, 2864,..."
1,1,66,1,"[2396, 2693, 2246, 463, 2426, 3456, 931, 1982,..."
2,2,207,1,"[1096, 587, 1544, 1219, 1289, 1880, 2978, 2520..."
3,3,208,1,"[1030, 3548, 54, 1367, 2561, 1971, 1284, 2553,..."
4,4,222,1,"[1021, 1603, 2924, 2911, 460, 2844, 1607, 1082..."


In [72]:
from eval_metrics import RecoMetrics

In [73]:
rm_metric = RecoMetrics()

In [75]:
rm_metric.set_interactions(positive_interactions_users, 
                           test_items, 
                           np.random.randint(1,101,len(positive_interactions_users)).tolist(),
                           negative_interactions_users, 
                           negative_items,
                           np.random.randint(1,101,len(negative_interactions_users)).tolist(),)

In [76]:
rm_metric._ranked_users_scores_df.head()

,users,items,scores,test_items,test_scores,rank
75,0,3610,100,25,47,1.0
49,0,3355,95,25,47,2.0
66,0,1313,92,25,47,3.0
69,0,3362,91,25,47,4.0
50,0,1230,90,25,47,5.0


In [77]:
rm_metric.cal_hit_ratio()

0.03195364238410596

In [78]:
deep_recommender = GIMFDeepRecommender(
    config=gmf_config, num_users=num_users, num_items=num_items)
# training
for epoch in range(3):
    logger.info("Epoch {}".format(epoch))
    train_data = sample_generator._prepare_epoch(
        gmf_config['num_negatives'], gmf_config['batch_size'])
    deep_recommender.train_epoch(train_data, epoch_num=epoch)


INFO:base_deep_recommender:Base Deep Recommender got the ocnfig {'alias': 'gmf_imf', 'num_epoch': 200, 'batch_size': 1024, 'optimizer': 'adam', 'adam_lr': '1e-3', 'latent_dim': 8, 'num_negatives': 4, 'l2_regularization': 0.1, 'use_cuda': True, 'model_dir': 'checkpoints/{}_Epoch{}_HR{:.4f}_NDCG{:.4f}.model'}
INFO:__main__:Epoch 0
INFO:data_processor:Epoch should be epoch number data is prepared via pandas
INFO:base_deep_recommender:starting training Epoch 0
INFO:base_deep_recommender:self.model.num_users is 6040
INFO:base_deep_recommender:Epoch 0 has batch 0 loss 0.6476732492446899
INFO:base_deep_recommender:Epoch 0 has batch 1 loss 0.6555219888687134
INFO:base_deep_recommender:Epoch 0 has batch 2 loss 0.6574348211288452
INFO:base_deep_recommender:Epoch 0 has batch 3 loss 0.6545922756195068
INFO:base_deep_recommender:Epoch 0 has batch 4 loss 0.6316311955451965
INFO:base_deep_recommender:Epoch 0 has batch 5 loss 0.6506476998329163
INFO:base_deep_recommender:Epoch 0 has batch 6 loss 0.647

INFO:base_deep_recommender:Epoch 0 has batch 107 loss 0.611520528793335
INFO:base_deep_recommender:Epoch 0 has batch 108 loss 0.6143566370010376
INFO:base_deep_recommender:Epoch 0 has batch 109 loss 0.6060458421707153
INFO:base_deep_recommender:Epoch 0 has batch 110 loss 0.6259253025054932
INFO:base_deep_recommender:Epoch 0 has batch 111 loss 0.6051655411720276
INFO:base_deep_recommender:Epoch 0 has batch 112 loss 0.5981574654579163
INFO:base_deep_recommender:Epoch 0 has batch 113 loss 0.602204442024231
INFO:base_deep_recommender:Epoch 0 has batch 114 loss 0.6134805083274841
INFO:base_deep_recommender:Epoch 0 has batch 115 loss 0.6029088497161865
INFO:base_deep_recommender:Epoch 0 has batch 116 loss 0.5929221510887146
INFO:base_deep_recommender:Epoch 0 has batch 117 loss 0.6074717044830322
INFO:base_deep_recommender:Epoch 0 has batch 118 loss 0.5904055237770081
INFO:base_deep_recommender:Epoch 0 has batch 119 loss 0.6058531999588013
INFO:base_deep_recommender:Epoch 0 has batch 120 loss

INFO:base_deep_recommender:Epoch 0 has batch 220 loss 0.5879117250442505
INFO:base_deep_recommender:Epoch 0 has batch 221 loss 0.573800802230835
INFO:base_deep_recommender:Epoch 0 has batch 222 loss 0.572537899017334
INFO:base_deep_recommender:Epoch 0 has batch 223 loss 0.5780613422393799
INFO:base_deep_recommender:Epoch 0 has batch 224 loss 0.584388017654419
INFO:base_deep_recommender:Epoch 0 has batch 225 loss 0.5800644159317017
INFO:base_deep_recommender:Epoch 0 has batch 226 loss 0.5725064277648926
INFO:base_deep_recommender:Epoch 0 has batch 227 loss 0.5757902264595032
INFO:base_deep_recommender:Epoch 0 has batch 228 loss 0.5664925575256348
INFO:base_deep_recommender:Epoch 0 has batch 229 loss 0.5739147067070007
INFO:base_deep_recommender:Epoch 0 has batch 230 loss 0.5714454650878906
INFO:base_deep_recommender:Epoch 0 has batch 231 loss 0.5836649537086487
INFO:base_deep_recommender:Epoch 0 has batch 232 loss 0.5760434865951538
INFO:base_deep_recommender:Epoch 0 has batch 233 loss 

INFO:base_deep_recommender:Epoch 0 has batch 333 loss 0.5707434415817261
INFO:base_deep_recommender:Epoch 0 has batch 334 loss 0.5684581995010376
INFO:base_deep_recommender:Epoch 0 has batch 335 loss 0.5605330467224121
INFO:base_deep_recommender:Epoch 0 has batch 336 loss 0.5525983572006226
INFO:base_deep_recommender:Epoch 0 has batch 337 loss 0.5523339509963989
INFO:base_deep_recommender:Epoch 0 has batch 338 loss 0.5625971555709839
INFO:base_deep_recommender:Epoch 0 has batch 339 loss 0.5651448965072632
INFO:base_deep_recommender:Epoch 0 has batch 340 loss 0.562407910823822
INFO:base_deep_recommender:Epoch 0 has batch 341 loss 0.5546643733978271
INFO:base_deep_recommender:Epoch 0 has batch 342 loss 0.5582224726676941
INFO:base_deep_recommender:Epoch 0 has batch 343 loss 0.5697442293167114
INFO:base_deep_recommender:Epoch 0 has batch 344 loss 0.5625013113021851
INFO:base_deep_recommender:Epoch 0 has batch 345 loss 0.5558773279190063
INFO:base_deep_recommender:Epoch 0 has batch 346 los

INFO:base_deep_recommender:Epoch 0 has batch 446 loss 0.544095516204834
INFO:base_deep_recommender:Epoch 0 has batch 447 loss 0.5569827556610107
INFO:base_deep_recommender:Epoch 0 has batch 448 loss 0.5492247343063354
INFO:base_deep_recommender:Epoch 0 has batch 449 loss 0.5686379075050354
INFO:base_deep_recommender:Epoch 0 has batch 450 loss 0.543337881565094
INFO:base_deep_recommender:Epoch 0 has batch 451 loss 0.5377354621887207
INFO:base_deep_recommender:Epoch 0 has batch 452 loss 0.5470110177993774
INFO:base_deep_recommender:Epoch 0 has batch 453 loss 0.5549781918525696
INFO:base_deep_recommender:Epoch 0 has batch 454 loss 0.5601523518562317
INFO:base_deep_recommender:Epoch 0 has batch 455 loss 0.5470683574676514
INFO:base_deep_recommender:Epoch 0 has batch 456 loss 0.5627253651618958
INFO:base_deep_recommender:Epoch 0 has batch 457 loss 0.5574398040771484
INFO:base_deep_recommender:Epoch 0 has batch 458 loss 0.5461042523384094
INFO:base_deep_recommender:Epoch 0 has batch 459 loss

...

INFO:base_deep_recommender:Epoch 2 has batch 4326 loss 0.5122531056404114
INFO:base_deep_recommender:Epoch 2 has batch 4327 loss 0.5369382500648499
INFO:base_deep_recommender:Epoch 2 has batch 4328 loss 0.5351822376251221
INFO:base_deep_recommender:Epoch 2 has batch 4329 loss 0.5263844728469849
INFO:base_deep_recommender:Epoch 2 has batch 4330 loss 0.5123077034950256
INFO:base_deep_recommender:Epoch 2 has batch 4331 loss 0.5202550888061523
INFO:base_deep_recommender:Epoch 2 has batch 4332 loss 0.5299601554870605
INFO:base_deep_recommender:Epoch 2 has batch 4333 loss 0.540547251701355
INFO:base_deep_recommender:Epoch 2 has batch 4334 loss 0.5308922529220581
INFO:base_deep_recommender:Epoch 2 has batch 4335 loss 0.5142128467559814
INFO:base_deep_recommender:Epoch 2 has batch 4336 loss 0.5300779342651367
INFO:base_deep_recommender:Epoch 2 has batch 4337 loss 0.500225305557251
INFO:base_deep_recommender:Epoch 2 has batch 4338 loss 0.5292483568191528
INFO:base_deep_recommender:Epoch 2 has b

INFO:base_deep_recommender:Epoch 2 has batch 4437 loss 0.5163415670394897
INFO:base_deep_recommender:Epoch 2 has batch 4438 loss 0.5128797292709351
INFO:base_deep_recommender:Epoch 2 has batch 4439 loss 0.5250149965286255
INFO:base_deep_recommender:Epoch 2 has batch 4440 loss 0.5388866662979126
INFO:base_deep_recommender:Epoch 2 has batch 4441 loss 0.5362921953201294
INFO:base_deep_recommender:Epoch 2 has batch 4442 loss 0.5172344446182251
INFO:base_deep_recommender:Epoch 2 has batch 4443 loss 0.5319802761077881
INFO:base_deep_recommender:Epoch 2 has batch 4444 loss 0.5103264451026917
INFO:base_deep_recommender:Epoch 2 has batch 4445 loss 0.5172607898712158
INFO:base_deep_recommender:Epoch 2 has batch 4446 loss 0.5250551700592041
INFO:base_deep_recommender:Epoch 2 has batch 4447 loss 0.5189831852912903
INFO:base_deep_recommender:Epoch 2 has batch 4448 loss 0.5146393775939941
INFO:base_deep_recommender:Epoch 2 has batch 4449 loss 0.5232914090156555
INFO:base_deep_recommender:Epoch 2 has

INFO:base_deep_recommender:Epoch 2 has batch 4548 loss 0.517881453037262
INFO:base_deep_recommender:Epoch 2 has batch 4549 loss 0.5047141909599304
INFO:base_deep_recommender:Epoch 2 has batch 4550 loss 0.52314692735672
INFO:base_deep_recommender:Epoch 2 has batch 4551 loss 0.5205065608024597
INFO:base_deep_recommender:Epoch 2 has batch 4552 loss 0.5196231603622437
INFO:base_deep_recommender:Epoch 2 has batch 4553 loss 0.5161042213439941
INFO:base_deep_recommender:Epoch 2 has batch 4554 loss 0.5275158882141113
INFO:base_deep_recommender:Epoch 2 has batch 4555 loss 0.5354210138320923
INFO:base_deep_recommender:Epoch 2 has batch 4556 loss 0.5134694576263428
INFO:base_deep_recommender:Epoch 2 has batch 4557 loss 0.5196221470832825
INFO:base_deep_recommender:Epoch 2 has batch 4558 loss 0.523138165473938
INFO:base_deep_recommender:Epoch 2 has batch 4559 loss 0.5363169312477112
INFO:base_deep_recommender:Epoch 2 has batch 4560 loss 0.5284276008605957
INFO:base_deep_recommender:Epoch 2 has bat

INFO:base_deep_recommender:Epoch 2 has batch 4659 loss 0.5109878778457642
INFO:base_deep_recommender:Epoch 2 has batch 4660 loss 0.5144543647766113
INFO:base_deep_recommender:Epoch 2 has batch 4661 loss 0.5100228786468506
INFO:base_deep_recommender:Epoch 2 has batch 4662 loss 0.5196335315704346
INFO:base_deep_recommender:Epoch 2 has batch 4663 loss 0.4993825852870941
INFO:base_deep_recommender:Epoch 2 has batch 4664 loss 0.5248137712478638
INFO:base_deep_recommender:Epoch 2 has batch 4665 loss 0.5194933414459229
INFO:base_deep_recommender:Epoch 2 has batch 4666 loss 0.5115306973457336
INFO:base_deep_recommender:Epoch 2 has batch 4667 loss 0.5150057077407837
INFO:base_deep_recommender:Epoch 2 has batch 4668 loss 0.5026247501373291
INFO:base_deep_recommender:Epoch 2 has batch 4669 loss 0.5246039628982544
INFO:base_deep_recommender:Epoch 2 has batch 4670 loss 0.5219176411628723
INFO:base_deep_recommender:Epoch 2 has batch 4671 loss 0.5148224830627441
INFO:base_deep_recommender:Epoch 2 has

INFO:base_deep_recommender:Epoch 2 has batch 4770 loss 0.5136374235153198
INFO:base_deep_recommender:Epoch 2 has batch 4771 loss 0.5301209688186646
INFO:base_deep_recommender:Epoch 2 has batch 4772 loss 0.4988532066345215
INFO:base_deep_recommender:Epoch 2 has batch 4773 loss 0.5413829684257507
INFO:base_deep_recommender:Epoch 2 has batch 4774 loss 0.5326874256134033
INFO:base_deep_recommender:Epoch 2 has batch 4775 loss 0.5144410133361816
INFO:base_deep_recommender:Epoch 2 has batch 4776 loss 0.5092169642448425
INFO:base_deep_recommender:Epoch 2 has batch 4777 loss 0.5430885553359985
INFO:base_deep_recommender:Epoch 2 has batch 4778 loss 0.5074506998062134
INFO:base_deep_recommender:Epoch 2 has batch 4779 loss 0.5187326669692993
INFO:base_deep_recommender:Epoch 2 has batch 4780 loss 0.5334950685501099
INFO:base_deep_recommender:Epoch 2 has batch 4781 loss 0.5221818685531616
INFO:base_deep_recommender:Epoch 2 has batch 4782 loss 0.5386975407600403
INFO:base_deep_recommender:Epoch 2 has